In [1]:
import sys

## This to get the peerless target star DataFrame for example purposes
#sys.path.append('/u/tdm/repositories/peerless/prediction')
#sys.path.append('/u/tdm/repositories/peerless')
#from targets import targets


import pandas as pd
targets = pd.read_hdf('targets.h5')

# The action is here. Depends on vespa & isochrones.
from exosyspop.populations import KeplerBinaryPopulation

In [2]:
pop = KeplerBinaryPopulation(targets, fB=0.4)

In [3]:
# Accessing secondary properties will initialize a secondary simulation,
# calling pop._generate_binaries().  The first time this is called, the
# secondary property regressors get trained.
pop.radius_B

dmag regressor trained, R2=0.99954097449
qR regressor trained, R2=0.999498769996


array([        nan,         nan,  0.67073112, ...,  0.63813378,
        0.28178712,         nan])

In [4]:
# subsequent calls are much faster; e.g.
pop._generate_binaries()
print(pop.radius_B)
%timeit pop._generate_binaries()

[        nan  0.21464393         nan ...,         nan         nan
         nan]
10 loops, best of 3: 97.4 ms per loop


In [5]:
# If physical accuracy is important, you can also choose to generate binary properties
# directly from the isochrone, but it's a factor of a few slower:
pop._generate_binaries(use_ic=True)
print(pop.radius_B)
%timeit pop._generate_binaries(use_ic=True)

[        nan         nan  0.74756056 ...,         nan  0.12412566
         nan]
1 loops, best of 3: 356 ms per loop


In [6]:
# Similarly, accessing orbital properties will generate them
pop.period

array([  1.19526129e+05,   2.50938408e+02,   8.93330939e+01, ...,
         5.22629197e+01,   3.49208202e+01,   1.73349147e+01])

# Synthetic observations

In [7]:
# Now, we can observe and see what we see.  This takes into account
# duty cycle & data span, as well as geometry.
obs = pop.observe()
print(len(obs))
print(obs.columns)
obs.head()

529
Index([u'host', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec'],
      dtype='object')


,host,period,ecc,w,inc,a,aR,b_pri,b_sec,k,...,T14_pri,T14_sec,T23_pri,T23_sec,dataspan,dutycycle,flux_ratio,n_pri,n_sec,phase_sec
0,609,6.229675,0.591941,0.820613,1.495747,1.163083e+12,19.646664,0.667758,1.687827,0.868603,...,0.099481,0.115405,0.000000,0.00000,1459.789,0.8751,0.515938,211,205,0.775200
1,878,1.755104,0.058031,2.750630,1.452779,4.514730e+11,8.200713,0.941505,0.984089,0.426689,...,0.072032,0.072538,0.000000,0.00000,1459.789,0.6987,0.015628,568,600,0.465852
2,1499,1.415073,0.438953,5.407751,1.295306,4.054972e+11,6.537987,2.165710,1.073855,0.446262,...,0.000000,0.046796,0.000000,0.00000,1459.789,0.8752,0.015501,0,884,0.687290
3,1517,1.169640,0.135898,3.344064,1.515084,3.446115e+11,3.836275,0.215561,0.204093,0.119424,...,0.110302,0.104648,0.085244,0.08098,1459.789,0.8755,0.000422,1092,1087,0.415472
4,1861,685.622170,0.000374,1.371500,1.569825,2.768710e+13,398.725272,0.387136,0.387420,0.909675,...,1.023177,1.023895,0.000000,0.00000,1459.789,0.8749,0.732362,1,2,0.500047


In [8]:
# This is pretty fast, even when generating a new population each time:
%timeit pop.observe(new=True)

1 loops, best of 3: 217 ms per loop


In [9]:
# Even faster if we only generate new orbits.
%timeit pop.observe(new_orbits=True)

10 loops, best of 3: 85.2 ms per loop


In [10]:
# So we can predict the expected number of observations pretty easily.
import numpy as np
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True)) for i in range(N)])
n_obs.mean(), n_obs.std()

(531.37, 22.693900061470263)

In [11]:
# Notice that the above does not yet have trapezoidal parameters.  There are two options to generate these.
# Either we can set the fit_trap parameter, as follows:
obs = pop.observe(fit_trap=True)
print(len(obs))
obs.columns

508


Index([u'host', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri',
       u'trap_depth_pri', u'trap_slope_pri', u'trap_dur_sec',
       u'trap_depth_sec', u'trap_slope_sec'],
      dtype='object')

In [12]:
# All things considered, this is still pretty fast if we just need to do it a few times:
%timeit pop.observe(fit_trap=True)

1 loops, best of 3: 2.46 s per loop


In [13]:
# However, this is pretty slow if we want to do inference.  To help with this, we can 
# tell it to train & use a regression.  Training only happens once; by default with 10,000 
# synthetic observations.  This takes a minute or so.
obs = pop.observe(regr_trap=True)
print(len(obs))
obs.columns

Depth trained: R2=0.998214479742
Duration trained: R2=0.99183295272
Slope trained: R2=0.990978838946
554


Index([u'host', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [14]:
# Subsequent calls are much faster
%timeit pop.observe(regr_trap=True)

10 loops, best of 3: 34.4 ms per loop


In [15]:
# Even generating a new stellar population & observing it is pretty quick
%timeit pop.observe(regr_trap=True, new=True)

1 loops, best of 3: 237 ms per loop


In [16]:
# Or again, you can just generate new orbits (rather than new binaries & new orbits)
%timeit pop.observe(regr_trap=True, new_orbits=True)

10 loops, best of 3: 108 ms per loop


In [17]:
# Generating the training data used for the trapezoid shape regression above used
# this function, which can be otherwise useful to sample >N random observations 
# from the existing population.  `trap_regr` defaults to `True` here.  
# This function also takes `new` or `new_orbits` keywords.
obs_pop = pop.get_N_observed(N=10000, new_orbits=True)
print(len(obs_pop))
obs_pop.columns

10439


Index([u'host', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'phase_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [18]:
# We can now look, e.g. at the expected number of single/double eclipsing systems:
query = '(n_pri < 3) & (n_sec < 3) & (n_pri==0 | n_sec==0)'
N = 100
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(6.2800000000000002, 2.2850820554194549)

In [19]:
# Try this again, this time using the empirical eccentricity distribution
# (as opposed to the beta distribution with default params)---eccentricity matters!
pop.ecc_empirical = True
n_obs = np.array([len(pop.observe(new_orbits=True).query(query)) for i in range(N)])
n_obs.mean(), n_obs.std()

(9.9299999999999997, 3.2007967758044251)